In [ ]:
from PIL import Image
import requests
import faiss
import json
from extract_features import mySigLipModel
import time

In [ ]:
class ImageIndex():
    def __init__(self):
        self.extractor = mySigLipModel()
        self.index = faiss.IndexFlatL2(self.extractor.shape)
        self.train_images_path = "./dataset/SBU_captioned_photo_dataset_urls.txt"
        self.train_images = open(self.train_images_path, "r").readlines()
    
    def indexing(self, index_path, image_path, start, end):
        image_urls = []

        print("start indexing...")
        start = time.time()

        for i, img_url in enumerate(self.train_images[start:end]):
            img_url = img_url.strip()
            try:
                frame = Image.open(requests.get(img_url, stream=True).raw)
            except:
                print(f"Failed to load image {i}: {img_url}")
                continue
            image_urls.append(img_url)
            embedding = self.extractor.get_image_embedding(frame)
            self.index.add(embedding)
            if i % 100 == 0:
                print(f"Indexed {i} images")

        end = time.time()
        print('Finish in ' + str(end - start) + ' seconds')
        faiss.write_index(self.index, index_path)

        with open(image_path, "w") as f:
            json.dump(image_urls, f, indent=2)

In [ ]:
# def indexing_image(index_path, image_path, start, end):
#     #load in the embedding extractor
#     extractor = mySigLipModel()
#     #load in indexer
#     index = faiss.IndexFlatL2(extractor.shape)
#     #load in image dataset
#     train_images_path = "./dataset/SBU_captioned_photo_dataset_urls.txt"
#     train_images = open(train_images_path, "r").readlines()

#     image_urls = []

#     print("start indexing...")
#     start = time.time()

#     for i, img_url in enumerate(train_images[start:end]):
#         img_url = img_url.strip()
#         try:
#             frame = Image.open(requests.get(img_url, stream=True).raw)
#         except:
#             print(f"Failed to load image {i}: {img_url}")
#             continue
#         image_urls.append(img_url)
#         embedding = extractor.get_image_embedding(frame)
#         # print(embedding.shape)
#         index.add(embedding)
#         # print(f"Added image {img_url} to index")
#         if i % 100 == 0:
#             print(f"Indexed {i} images")

#     end = time.time()
#     print('Finish in ' + str(end - start) + ' seconds')
#     faiss.write_index(index, index_path)

#     with open(image_path, "w") as f:
#         json.dump(image_urls, f, indent=2)

In [ ]:
#load in storage path
# feature_root = './indexed_dataset/'
index_path = "siglip-image-index-90k.bin"
image_path = "siglip_image_urls-90k.json"

indexer = ImageIndex()
indexer.indexing(index_path, image_path, 10000, 90000)

from google.colab import files
files.download('siglip-image-index-90k.bin')
files.download('siglip_image_urls-90k.json')
# files.download('file.txt')